[Oregon Curriculum Network](http://4dsolutions.net/ocn/)<br/>
[School of Tomorrow](School_of_Tomorrow.ipynb)


# MAKING SHAPES

![ball_nest.gif](ball_nest.gif)

How do I make these animated GIFs such as the one above? As you might expect: there's a pipeline, a sequence of steps. 

First comes a Python script that, in turn, depends on several already-written modules, some by me, some by others. 

Then comes the POV-Ray step, wherein I "render" (turn into a picture) what the Python script has put out: a file in Scene Description Language (file extension: pov). 

Finally, I've been using a 3rd party package named Fiji, a repackaging of ImageJ (written in Java) to crop and stitch the stills (png files) into a single GIF that loops through its two or more frames automatically.

In [76]:
from flextegrity import pov_header, Cuboctahedron, Cube, Octahedron, RT
from flextegrity import Tetrahedron, InvTetrahedron, RD, PD, Icosahedron, Mite
from flextegrity import Edge, draw_edge, draw_poly, draw_vert, half, ORIGIN, PHI
from qrays import Qvector, Vector, A, B, C, D

import numpy as np
import sympy as sy
from sympy import sqrt as rt2, sin, cos
from mpmath import radians

from itertools import permutations
g = permutations((2,1,1,0))
UNIQUE = {p for p in g}  # set comprehension

IVM_DIRS = {Qvector(x) for x in UNIQUE}

Svol = (PHI **-5)/2  
Evol = (rt2(2)/8) * (PHI ** -3)

sfactor = Svol/Evol

CLOSEUP = \
"""
   
// perspective (default) camera
camera {
  location  <3, 0.1, 0.2>
  rotate    <35, 20, 33.0>
  look_at   <0.0, 0.0,  0.0>
  right     x*image_width/image_height
}

"""

That big fat import above, plus defining of constants, so-called globals to be used further on (scripts run top to bottom), is how I usually start my various rendering projects. I know what tools I'm gonna want: quadrays, polyhedrons, trig functions and so on.

In this Notebook, my plan is to make an animated GIF with the following frames:

* the Icosahedron, already defined, and a reference IVM ball, start the show
* the PD (Icosa's dual) gets added
* the RT (their sum total) appears (their "begot")
* the RT shrinks down to embrace the IVM ball more tightly
* show the RT shrink-wrapping the IVM ball with no other guys in view

That's it for now. We could keep going. I'm tempted to shrink that RT just a tad further, from volume ~5.0078 to volume 5 exactly, then to expand it (volume-wise) by 5/2 to volume 7.5 exactly, where its corners would precisely align with the RD's of volume six. But I'll save that for another day. Let's get our cast of characters on the scene then.

In [82]:
def test7():
    global ic, pd, rt, rt_e
    
    ic    = Icosahedron()    # edges D
    ic.edge_radius = 0.03

    pd  = PD()               # Icosa's dual
    pd.edge_radius = 0.03

    rt  = RT()               # their "begot" 
    rt.edge_radius = 0.03
    
    rt_e    = rt * (1/PHI)          
    rt_e.edge_radius = 0.03

test7()  # run the above function

That's enough code to instantiate our cast of polyhedra, minus the IVM ball which we think of as an enlarged vertex and will add when it's time to actually generate our scene description language.

The code above is sufficient to give us an initial volumes table:

In [34]:
ic.volume

5*sqrt(2)*(1/2 + sqrt(5)/2)**2

In [35]:
str(ic.volume)

'5*sqrt(2)*(1/2 + sqrt(5)/2)**2'

In [16]:
ic.volume.evalf()

18.5122958682192

In [74]:
print(
"""
Icosahedron: {0:>40}   {4:>12.11g}
PD:          {1:>40}   {5:>12.11g}
RT:          {2:>40}   {6:>12.11g}
RT_E:        {3:>40}   {7:>12.10g}
""".format(str(ic.volume), str(pd.volume), str(rt.volume), str(rt_e.volume), 
           ic.volume.evalf(), pd.volume.evalf(), rt.volume.evalf(), rt_e.volume.evalf()))


Icosahedron:           5*sqrt(2)*(1/2 + sqrt(5)/2)**2   18.512295868
PD:              3*sqrt(2)*(1 + (1/2 + sqrt(5)/2)**2)   15.350018208
RT:                                        15*sqrt(2)   21.213203436
RT_E:                 15*sqrt(2)/(1/2 + sqrt(5)/2)**3    5.007758031



The Icosahedron, already defined, and a reference IVM ball, start the show...

In [77]:
with open("genesis_1.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076692644/in/dateposted/" title="Frame 1: Icosa + IVM Ball"><img src="https://live.staticflickr.com/65535/55076692644_e9b2bde5fd.jpg" width="500" height="436" alt="Frame 1: Icosa + IVM Ball"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

The PD (Icosa's dual) gets added...

In [78]:
with open("genesis_2.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076435251/in/dateposted/" title="Frame 2: Icosa + PD (dual)"><img src="https://live.staticflickr.com/65535/55076435251_1302f67720.jpg" width="500" height="436" alt="Frame 2: Icosa + PD (dual)"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

the RT (their sum total) appears (their "begot")...

In [79]:
with open("genesis_3.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_poly(rt, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076692634/in/dateposted/" title="Frame 3: Icosa + PD Beget RT"><img src="https://live.staticflickr.com/65535/55076692634_61ac44f5a0.jpg" width="500" height="436" alt="Frame 3: Icosa + PD Beget RT"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

The RT shrinks down to embrace the IVM ball more tightly...

In [80]:
with open("genesis_4.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(ic, T)
    draw_poly(pd, T)
    draw_poly(rt_e, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076640078/in/dateposted/" title="Frame 4: RT Shrinks by 1/PHI In All Linear DImensions"><img src="https://live.staticflickr.com/65535/55076640078_6ae432e5d1.jpg" width="500" height="436" alt="Frame 4: RT Shrinks by 1/PHI In All Linear DImensions"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Show the RT shrink-wrapping the IVM ball with no other guys in view

In [81]:
with open("genesis_5.pov", "w") as T:
    T.write(pov_header) 
    T.write(CLOSEUP)
    draw_poly(rt_e, T)
    draw_vert(ORIGIN, "T_Stone18", half, T, texture=True)

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/55076442931/in/dateposted/" title="Frame 5: RT_E Hugs IVM Ball"><img src="https://live.staticflickr.com/65535/55076442931_f2f02d5786.jpg" width="500" height="436" alt="Frame 5: RT_E Hugs IVM Ball"/></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

With all five frames rendered individually, we're ready to stitch them together in a final product...

![genesis_story.gif](genesis_story.gif)